# Conect TF-Serving with gRFC

This notebook shows steps to communicate tf-serving with gRFC.

For that we first need to install some libraries like `grpcio` client (comes with tf package) and `tensorflow-serving-api`.

In [23]:
!pip install tensorflow-serving-api==2.7.0

Another library we need for image preprocessing is `keras-image-helper`, this will make our gateway service lightweight without the need to tensorflow package.

In [24]:
!pip install keras-image-helper

Now we need implement few steps by importing libraries to convert image into specific format (pad_batch).

In [25]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [26]:
# Local host port
host = 'localhost:8500'

# Create channel
channel = grpc.insecure_channel(host)

# Use channel to connect to prediction service
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel) # PredictionServiceStub is used to invoke localhost services
stub

Now we'll using `stub` to make image predictions on localhost.

In [27]:
from keras_image_helper import create_preprocessor

In [28]:
# Create input image preprocessor
preprocessor = create_preprocessor('xception', target_size=(229, 229))

In [29]:
# Image url
img_url = 'http://bit.ly/mlbookcamp-pants'

# Preprocess image
X = preprocessor.from_url(img_url)
X.shape

(1, 229, 229, 3)

In [30]:
X

array([[[[-0.09803921, -0.1372549 , -0.18431371],
         [-0.09803921, -0.1372549 , -0.18431371],
         [-0.08235294, -0.12156862, -0.16862744],
         ...,
         [-0.02745098, -0.02745098, -0.09019607],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.11372548, -0.11372548, -0.17647058]],

        [[-0.09019607, -0.12941176, -0.17647058],
         [-0.09019607, -0.12941176, -0.17647058],
         [-0.06666666, -0.10588235, -0.15294117],
         ...,
         [-0.02745098, -0.02745098, -0.09019607],
         [-0.03529412, -0.03529412, -0.09803921],
         [-0.09803921, -0.09803921, -0.1607843 ]],

        [[-0.09803921, -0.1372549 , -0.18431371],
         [-0.09803921, -0.1372549 , -0.18431371],
         [-0.08235294, -0.12156862, -0.16862744],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.03529412, -0.03529412, -0.09803921],
         [-0.08235294, -0.08235294, -0.14509803]],

        ...,

        [[-0.827451  , -0.8666667 , -0

Now we need to turn the numpy feature `X` into tensorflow proto.

In [31]:
# Function to convert numpy array to protobuf
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

The image is processed and this is what we want to send to our prediction service.

Now we need to prepare the request in json format using `predict_pb2`.

In [32]:
# Create pb request
pb_request = predict_pb2.PredictRequest()

# Provide model name
pb_request.model_spec.name = 'clothing-model'

# Signature name
pb_request.model_spec.signature_name = 'serving_default'

# Specify Inputs name and assign protobuf feature to it
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [33]:
# View pb request
pb_request

model_spec {
  name: "clothing-model"
  signature_name: "serving_default"
}
inputs {
  key: "input_8"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 229
      }
      dim {
        size: 229
      }
      dim {
        size: 3
      }
    }
    tensor_content: "\310\310\310\275\214\214\014\276\274\274<\276\310\310\310\275\214\214\014\276\274\274<\276\250\250\250\275\370\370\370\275\254\254,\276\320\320P\275\270\270\270\275\214\214\014\276\260\2600\275\250\250\250\275\204\204\004\276\220\220\020\275\230\230\230\275\370\370\370\275\200\200\200\273\260\2600\275\270\270\270\275\000\301@<\340\340\340\274\230\230\230\275\000\301@<\340\340\340\274\230\230\230\275\300\240\240<\240\240\240\274\260\2600\275\340\320P=\000\301@<\300\300@\274\300\270\270=\340\320P=\000\341\340<\340\330\330=\220\210\210=\300\2600=\360\350\350=\240\230\230=\340\320P=\210\204\004>\300\270\270=\220\210\210=\230\224\024>\340\330\330=\260\250\250=\270\2644>\

We have prepared the request, let's execute it using stub we created earlier.

In [34]:
pb_response = stub.Predict(pb_request, timeout=20.0) # specify timeout in case service is not responsive 

In [35]:
# Check the output response
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -2.4946017265319824
    float_val: -8.939651489257812
    float_val: -2.9435999393463135
    float_val: -0.9738842248916626
    float_val: 15.57507610321045
    float_val: -2.8537731170654297
    float_val: -7.355284690856934
    float_val: 4.215627670288086
    float_val: -4.100409507751465
    float_val: -9.35374641418457
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [36]:
# Get the actual value of the output (predictions), it returns python list not numpy array,
# therefore, we don't need to do any conversion to make requests
preds = pb_response.outputs['dense_7'].float_val
preds

[-2.4946017265319824, -8.939651489257812, -2.9435999393463135, -0.9738842248916626, 15.57507610321045, -2.8537731170654297, -7.355284690856934, 4.215627670288086, -4.100409507751465, -9.35374641418457]

In [37]:
# List of class names
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [38]:
# Check the predictions with the corresponding class names
dict(zip(classes, preds))

{'dress': -2.4946017265319824,
 'hat': -8.939651489257812,
 'longsleeve': -2.9435999393463135,
 'outwear': -0.9738842248916626,
 'pants': 15.57507610321045,
 'shirt': -2.8537731170654297,
 'shoes': -7.355284690856934,
 'shorts': 4.215627670288086,
 'skirt': -4.100409507751465,
 't-shirt': -9.35374641418457}